In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn.model_selection import train_test_split
import re
import spacy
nlp = spacy.load('en_core_web_sm')

### Create files to read to doccano

In [75]:
for file in os.listdir('../../../Files/Submissions/train/'):
    if file.endswith('.pickle'):
        df = pd.read_pickle(os.path.join('../../../Files/Submissions/train', file))
        df['text'] = df['title'] + ' ' + df['selftext']
        df['label'] =  0
        df['text'] = df['text'].astype(str)
        df.to_csv(os.path.join('../../../Files/Submissions/train/', ( file[:-7] + '.csv')), sep='|',header=True, index=False)

In [45]:

files = os.listdir('../../../Files/Submissions/')
files = [file for file in files if file.endswith('.pickle')]
df2 = pd.read_pickle(os.path.join('../../../Files/Submissions/', files.pop(0)))
for file in files:
    temp = pd.read_pickle(os.path.join('../../../Files/Submissions/', file))
    temp = temp.sample(frac=0.1)
    df2 = df2.append(temp)
    


/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_3205/2192292866.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(temp)
/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_3205/2192292866.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(temp)
/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_3205/2192292866.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(temp)
/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_3205/2192292866.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(temp)
/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000

In [48]:
df2['text'] = df2['title'] + ' ' + df2['selftext']

In [49]:
df3 = df2.sample(4000)

In [50]:
df3.to_csv('../../../Files/Submissions/train/C1Candidates.csv', sep='|',header=True, index=False)

## Classifier I

In [2]:
df = pd.read_csv('../../../Files/Submissions/train/all.csv', sep=',')

In [4]:
df_ext = pd.read_csv('../../../Files/Submissions/train/all 3.csv', sep=',')

In [6]:
df = pd.concat([df, df_ext], ignore_index=True)

In [10]:
df_ext.groupby('label').count()

,id,text,author,created_utc,num_comments,permalink,score,selftext,subreddit,title,url
label,,,,,,,,,,,
Anti-Vaccine,186,186,186,186,186,186,186,88,186,186,186
Neutral,112,112,112,112,112,112,112,22,112,112,112
Non-Covid,120,120,120,120,120,120,120,11,120,120,120
Pro-Vaccine,167,167,167,167,167,167,167,145,167,167,167


In [13]:
df.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments
label,,,,,,,,,,,,
0.0,272,272,272,272,272,272,38,272,272,272,272,0
1.0,2304,2304,2304,2304,2304,2304,775,2304,2304,2304,2304,2


In [11]:
dicts = {'Anti-Vaccine': 1, 'Neutral' : 1, 'Pro-Vaccine': 1, 'Non-Covid': 0}

In [12]:
df = df.replace({'label' : dicts})

In [14]:
df4 = pd.read_csv('../../../Files/Submissions/train/all 2.csv')

In [15]:
df4 = df4.replace({'label' : dicts})

In [16]:
df4.drop(['cleanTitle', 'cleanText', 'index'], axis=1, inplace=True)

In [17]:
df5 = df[df['label'] == 1]

In [18]:
train1, test1 = train_test_split(df5, test_size=0.2, random_state=42)

In [19]:
train1 = pd.concat([train1, df[df['label'] == 0]])

In [20]:
train1.drop('Comments',axis=1, inplace=True)

In [22]:
train1.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments
label,,,,,,,,,,,
0.0,272,272,272,272,272,272,38,272,272,272,272
1.0,1843,1843,1843,1843,1843,1843,615,1843,1843,1843,1843


In [23]:
1843-272

1571

In [24]:
df6 = df4.sample(1571)

In [25]:
df7 = pd.concat([df4, df6, df6]).drop_duplicates(keep=False)

In [26]:
train1 = pd.concat([train1, df6])
test1 = pd.concat([test1, df7])

In [27]:
test1 = test1[test1['label'].notna()]

In [28]:
train1 = train1[train1['label'].notna()]

In [29]:
test1.drop('Comments', axis=1, inplace=True)

In [30]:
train1 = train1.sample(frac=1).reset_index(drop=True)
test1 = test1.sample(frac=1).reset_index(drop=True)

In [32]:
test1.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments
label,,,,,,,,,,,
0.0,2124,2124,2124,2124,2124,2124,296,2124,2124,2124,2124
1.0,461,461,461,461,461,461,160,461,461,461,461


## Classifier II

In [36]:
df = pd.read_csv('../../../Files/Submissions/train/all.csv', sep=',')

In [37]:
df_ext = pd.read_csv('../../../Files/Submissions/train/all 3.csv', sep=',')

In [38]:
df = pd.concat([df, df_ext], ignore_index=True)

In [39]:
dicts = {'Anti-Vaccine': 0, 'Neutral' : 1, 'Pro-Vaccine': 2}

In [40]:
df = df.replace({'label' : dicts})

In [41]:
df.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments
label,,,,,,,,,,,,
0,722,722,722,722,722,722,302,722,722,722,722,0
1,885,885,885,885,885,885,177,885,885,885,885,2
2,697,697,697,697,697,697,296,697,697,697,697,0
Non-Covid,272,272,272,272,272,272,38,272,272,272,272,0


In [42]:
df2 = df[(df['label'] != "Non-Covid") & (df['label'].notna())]

In [43]:
len(df2)

2304

In [44]:

train2, test2 = train_test_split(df2, test_size=0.2, random_state=42, stratify=df2['label'])

In [45]:
train2.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments
label,,,,,,,,,,,,
0,577,577,577,577,577,577,245,577,577,577,577,0
1,708,708,708,708,708,708,144,708,708,708,708,1
2,558,558,558,558,558,558,237,558,558,558,558,0


In [46]:
len(train2)

1843

In [47]:
test2.groupby(by='label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments
label,,,,,,,,,,,,
0,145,145,145,145,145,145,57,145,145,145,145,0
1,177,177,177,177,177,177,33,177,177,177,177,1
2,139,139,139,139,139,139,59,139,139,139,139,0


In [ ]:
len(test2)

368

## Text Preprocessing

In [33]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleantext = re.sub(r'[^\w\s]', '', sentence)
    rem_url=re.sub(r'http\S+', '[URL]',cleantext)
    rem_num = re.sub('[0-9]+', '[NUM]', rem_url)
    lemma_words = [token.lemma_ for token in nlp(rem_num) if not token.is_stop]
    return " ".join(lemma_words)

In [34]:
train1['cleanText']=train1['text'].map(lambda s:preprocess(s)) 
test1['cleanText']=test1['text'].map(lambda s:preprocess(s)) 

In [35]:
train1.to_csv('../../../Files/Submissions/train/train1_v2.csv', sep='|',header=True, index=False)
test1.to_csv('../../../Files/Submissions/train/test1_v2.csv', sep='|',header=True, index=False)

In [48]:
train2['cleanText']=train2['text'].map(lambda s:preprocess(s)) 
test2['cleanText']=test2['text'].map(lambda s:preprocess(s)) 

In [49]:
train2.to_csv('../../../Files/Submissions/train/train2_v2.csv', sep='|',header=True, index=False)
test2.to_csv('../../../Files/Submissions/train/test2_v2.csv', sep='|',header=True, index=False)